# 优先级队列_Priority_Queue

## 基本实现
将任务的优先级与任务组成tuple加入最小堆，堆中会按照tuple的第一位也就是优先级进行排序，从而实现了优先级队列。

In [ ]:
class Array(object):
    def __init__(self, size=32):  # 关键属性：分配空间和存储单位（使用列表的单个元素作为一个存储单位）
        self._size = size
        self._items = [None] * size

    def __getitem__(self, index):  # Called to implement evaluation of self[index]实现下标访问.
        return self._items[index]

    def __setitem__(self, index, value):  # Called to implement assignment to self[index].
        self._items[index] = value

    def __len__(self):
        return self._size

    def clear(self, value=None):
        for i in range(len(self._items)):
            self._items[i] = value

    def __iter__(self):
        for item in self._items:
            yield item


class MinHeap(object):
    def __init__(self, maxsize=None):
        self.maxsize = maxsize
        self._elements = Array(maxsize)
        self._count = 0

    def __len(self):
        return self._count

    def add(self, value):
        if self._count >= self.maxsize:
            raise Exception('full')
        self._elements[self._count] = value
        self._count += 1
        self._siftup(self._count - 1)

    def _siftup(self, ndx):
        if ndx > 0:
            parent = (ndx - 1) // 2
            if self._elements[ndx] < self._elements[parent]:
                self._elements[ndx], self._elements[parent] = self._elements[parent], self._elements[ndx]
                self._siftup(parent)

    def extract(self):
        if self._count <= 0:
            raise Exception('empty')
        value = self._elements[0]
        self._count -= 1
        self._elements[0] = self._elements[self._count]
        self._siftdown(0)
        return value

    def _siftdown(self, ndx):
        left = (ndx * 2) + 1
        right = (ndx * 2) + 2
        # 找出当前节点及左右子节点中的最小值，与当前节点交换位置，并递归地对换下去的节点执行siftdown操作
        smallest = ndx
        if left < self._count and self._elements[left] < self._elements[smallest] and right < self._count and \
                self._elements[left] <= self._elements[right]:
            smallest = left
        elif left < self._count and self._elements[left] < self._elements[smallest] and right >= self._count:
            smallest = left
        elif right < self._count and self._elements[right] < self._elements[smallest]:
            smallest = right
        if smallest != ndx:
            self._elements[ndx], self._elements[smallest] = self._elements[smallest], self._elements[ndx]
            self._siftdown(smallest)

# 将任务的优先级与任务组成tuple加入最小堆，堆中会按照tuple的第一位也就是优先级进行排序，从而实现了优先级队列。
class PriorityQueue(object):
    def __init__(self, maxsize):
        self.maxsize = maxsize
        self._minheap = MinHeap(maxsize)

    def push(self, priority, value):
        # 注意这里把这个 tuple push 进去，python 比较 tuple 从第一个开始比较
        # 这样就实现了按照优先级排序
        entry = (priority, value)
        self._minheap.add(entry)  # 入队的时候会根据 priority 维持堆的特性

    def pop(self, with_priority=False):
        entry = self._minheap.extract()
        if with_priority:
            return entry
        else:
            return entry[1]

    def is_empty(self):
        return self._minheap._count == 0

## 进阶实现
优先级队列除了以上的基本操作之外，还有一下的操作需要考虑：
- 排序的稳定性：如何将两个优先级相等的任务加入到队列的顺序返回？
- 在优先级相同且没有默认的任务比较顺序时元组的比较会中断。
- 如果一个任务的优先级改变了，如何将任务在堆中移动到新的位置？
- 或者如果一个等待中的任务需要删除，应该如何找到任务并从队列中移除？

### 解决方案
#### 优先级比较问题
将任务条目作为三元素元组保存，分别为优先级、统计条目以及任务（priority，an entry count, and the task）, 每个任务的条目计数都不相同，因此仅仅对元组的前两项就可以完成所有的优先级条目的比较。

#### 任务的优先级更新与删除
解决该问题的关键是如何在队列中找到特定的任务，可以通过一个指向优先级队列内一个条目的字典来寻找任务（用任务名称映射优先级条目的对象）。

删除条目或者改变条目的优先级很困难，因为它会打破堆的固定特性，一个可能的解决方法是将条目任务标记为已删除（***需要将表示优先级条目的元组改为列表*），然后添加一个修改过的优先级的新条目。

注意在弹出优先级任务判断时判断任务是否为RTEMOVED，若是则循环弹出下一个最小优先级任务。

In [ ]:
import heapq
import itertools


class PriorityQueue(object):
    REMOVED = '<removed-task>'  # 被删除任务的占位字符

    def __init__(self):
        self.pq = []  # 初始化heapq所使用的list
        self.entry_finder = {}  # 任务到优先级条目的映射
        self.counter = itertools.count()  # 唯一计数器

    def add_task(self, task, priority=0):
        """添加一个新任务或者更新一个已存在任务的优先级"""
        if task in self.entry_finder:
            self.remove_task(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heapq.heappush(self.pq, entry)

    def remove_task(self, task):
        """将一个已存在的任务标记为REMOVED，若未找到Raise KeyError。"""
        entry = self.entry_finder.pop(task)
        entry[-1] = PriorityQueue.REMOVED

    def pop_task(self):
        """删除并返回最小优先级任务，如果队列已空Raise KeyError"""
        while self.pq:  # 循环直到弹出值不为REMOVED的task才返回
            priority, count, task = heapq.heappop(self.pq)
            if task is not PriorityQueue.REMOVED:
                del self.entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')

    def is_empty(self):
        return len(self.entry_finder) == 0